Проверяем версию питона из виртуального окружения

In [1]:
!python --version

Python 3.9.6


Устанавливаем зависимости

In [30]:
!pip install torch==2.1.0 pytorch-lightning==2.1.3 pytorch-forecasting==1.3.0 pytorch-tabular --force-reinstall

  Obtaining dependency information for torch==2.1.0 from https://files.pythonhosted.org/packages/4c/53/f1e58e147df8601c963df4b15045631f7e3d3caa5973bdf4e54a5cf6834e/torch-2.1.0-cp39-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for pytorch-lightning==2.1.3 from https://files.pythonhosted.org/packages/f6/95/a662bd12b255ee3b59c975277c67398410b5810ee72ff11e34134e10cc9a/pytorch_lightning-2.1.3-py3-none-any.whl.metadata
  Using cached pytorch_lightning-2.1.3-py3-none-any.whl.metadata (21 kB)
  Obtaining dependency information for pytorch-forecasting==1.3.0 from https://files.pythonhosted.org/packages/29/9c/112553c4404014b5931b014cfe475cb4c4b96adf0dc23a1ac1f3b9337cba/pytorch_forecasting-1.3.0-py3-none-any.whl.metadata
  Using cached pytorch_forecasting-1.3.0-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for pytorch-tabular from https://files.pythonhosted.org/packages/2c/b2/dfd199d03a351f9558dbdb0876444d955b7fe2231b9e82acc55fee1c29fb/pytorch_tabu

Импортируем пакеты

In [5]:
import pandas as pd
import numpy as np
import torch

Загружаем данные из csv

In [6]:
df = pd.read_csv('data/SANDUSDT.csv', sep=',', decimal='.', index_col='datetime', parse_dates=['datetime'])
df.head()

,Open,High,Low,Volume,N,open_close_delta,high_low_delta,sin_hour,cos_hour,sin_day_of_week,cos_day_of_week
datetime,,,,,,,,,,,
2020-08-14 13:00:00,0.008333,0.094710,0.008333,48759746.0,5422.0,0.071667,0.086377,-0.258819,-0.965926,-0.433884,-0.900969
2020-08-14 13:01:00,0.080000,0.080005,0.062936,20299537.0,2869.0,-0.014000,0.017069,-0.258819,-0.965926,-0.433884,-0.900969
2020-08-14 13:02:00,0.066000,0.071002,0.064000,19623360.0,2617.0,0.001685,0.007002,-0.258819,-0.965926,-0.433884,-0.900969
2020-08-14 13:03:00,0.067685,0.070659,0.066497,14743801.0,2179.0,0.002315,0.004162,-0.258819,-0.965926,-0.433884,-0.900969
2020-08-14 13:04:00,0.070000,0.073868,0.070000,15576142.0,2055.0,0.001758,0.003868,-0.258819,-0.965926,-0.433884,-0.900969


Копируем данные чтобы избежать ошибок при изменении `df`

In [7]:
df = df.copy()

Проверяем типы колонок

In [8]:
print(df.dtypes)

Open                float64
High                float64
Low                 float64
Volume              float64
N                   float64
open_close_delta    float64
high_low_delta      float64
sin_hour            float64
cos_hour            float64
sin_day_of_week     float64
cos_day_of_week     float64
dtype: object


Данные неравномерные, поэтому заполняем недостающие данные

In [9]:
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df.resample('1min').asfreq()
df = df.infer_objects(copy=False)

# Интерполируем только числовые колонки
numeric_cols = df.select_dtypes(include=["number"]).columns
df[numeric_cols] = df[numeric_cols].interpolate(method="linear")

df["datetime"] = df.index

Преобразуем `datetime` в числовой индекс

In [10]:
df["time_idx"] = (df.index - df.index[0]).total_seconds() // 60
df["time_idx"] = df["time_idx"].astype(int)
df.head()

,Open,High,Low,Volume,N,open_close_delta,high_low_delta,sin_hour,cos_hour,sin_day_of_week,cos_day_of_week,datetime,time_idx
datetime,,,,,,,,,,,,,
2020-08-14 13:00:00,0.008333,0.094710,0.008333,48759746.0,5422.0,0.071667,0.086377,-0.258819,-0.965926,-0.433884,-0.900969,2020-08-14 13:00:00,0
2020-08-14 13:01:00,0.080000,0.080005,0.062936,20299537.0,2869.0,-0.014000,0.017069,-0.258819,-0.965926,-0.433884,-0.900969,2020-08-14 13:01:00,1
2020-08-14 13:02:00,0.066000,0.071002,0.064000,19623360.0,2617.0,0.001685,0.007002,-0.258819,-0.965926,-0.433884,-0.900969,2020-08-14 13:02:00,2
2020-08-14 13:03:00,0.067685,0.070659,0.066497,14743801.0,2179.0,0.002315,0.004162,-0.258819,-0.965926,-0.433884,-0.900969,2020-08-14 13:03:00,3
2020-08-14 13:04:00,0.070000,0.073868,0.070000,15576142.0,2055.0,0.001758,0.003868,-0.258819,-0.965926,-0.433884,-0.900969,2020-08-14 13:04:00,4


Добавляем метку серии

In [11]:
df["series_id"] = "SANDUSDT"
df = df.reset_index(drop=True)
#df.index = df["time_idx"]
type(df.index)

pandas.core.indexes.range.RangeIndex

Проверяем есть ли пустые данные

In [12]:
df.isna().sum()

Open                0
High                0
Low                 0
Volume              0
N                   0
open_close_delta    0
high_low_delta      0
sin_hour            0
cos_hour            0
sin_day_of_week     0
cos_day_of_week     0
datetime            0
time_idx            0
series_id           0
dtype: int64

Конфигурируем параметры

In [13]:
max_encoder_length = 60
max_prediction_length = 10

Подготоваливаем тренировочные данные

In [14]:
# Разделение на train/test
train_cutoff = df["time_idx"].max() - max_prediction_length
df_train = df[df["time_idx"] <= train_cutoff]

# Сбрасываем индексы, т.к. после разделения на тестовые данные он вернулся в обычный Index, вместо RangeIndex ожидаемого TimeSeriesDataSet
df_train = df_train.reset_index(drop=True)
type(df_train.index)

pandas.core.indexes.range.RangeIndex

Подготоваливаем данные для проверки

In [15]:
#df_val = df[df["time_idx"] > train_cutoff] # в таком случае TimeSeriesDataSet не построится и будет ошибка

min_val_idx = df["time_idx"].max() - (max_prediction_length + max_encoder_length)
df_val = df[df["time_idx"] >= min_val_idx].copy()
type(df_train.index)

pandas.core.indexes.range.RangeIndex

Подготавливаем TimeSeriesDataSet

In [16]:
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer

training = TimeSeriesDataSet(
    df_train,
    time_idx="time_idx",
    target="Open",
    group_ids=["series_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["series_id"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["Open"],
    target_normalizer=GroupNormalizer(groups=["series_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# Validation dataset
validation = TimeSeriesDataSet.from_dataset(training, df_val)

len(training), len(validation)

(461875, 2)

!Доработать: Получается всего 2 окна validation, для теста оставлю так, но нужно больше. Количество данных вроде позволяет

Создаем Dataloader-ы

In [17]:
train_dataloader = training.to_dataloader(train=True, batch_size=64, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=64, num_workers=0)

In [18]:
import torch
import pytorch_lightning as pl
import pytorch_forecasting as pf

print(torch.__version__)     # 2.1.0
print(pl.__version__)        # 2.1.3
print(pf.__version__)        # 1.3.0

2.1.0
2.1.3
1.3.0


Создаем модель DeepAR

In [33]:
from pytorch_forecasting.models import DeepAR

model = DeepAR.from_dataset(
    training,
    learning_rate=1e-3,
    hidden_size=64,        # размер GRU-состояния
    rnn_layers=2,          # количество GRU-слоёв
    dropout=0.1,
)

type(model)

/Users/ilyashulepov/Projects/ml_labs/lab5/.venv/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/ilyashulepov/Projects/ml_labs/lab5/.venv/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


pytorch_forecasting.models.deepar._deepar.DeepAR

In [27]:
print(model.__module__)

pytorch_forecasting.models.deepar._deepar


In [22]:
from pytorch_lightning import LightningModule
isinstance(model, LightningModule) # Должно быть True

False

Trainer и обучение

In [21]:
from pytorch_lightning import Trainer

trainer = Trainer(
    max_epochs=10,            # или больше
    gradient_clip_val=0.1,
    accelerator="auto",       # автоматически CPU или GPU
    log_every_n_steps=10
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `DeepAR`

In [ ]:
import lightning.pytorch as pl

trainer = pl.Trainer(
    max_epochs=10,            # или больше
    gradient_clip_val=0.1,
    accelerator="auto",       # автоматически CPU или GPU
    log_every_n_steps=10
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name                   | Type                   | Params | Mode 
--------------------------------------------------------------------------
0 | loss                   | NormalDistributionLoss | 0      | train
1 | logging_metrics        | ModuleList             | 0      | train
2 | embeddings             | MultiEmbedding         | 1      | train
3 | rnn                    | LSTM                   | 52.0 K | train
4 | distribution_projector | Linear                 | 130    | train
--------------------------------------------------------------------------
52.1 K    Trainable params
0         Non-trainable params
52.1 K    Total params
0.208     Total estimated model params size (MB)
12        Modules in train mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/ilyashulepov/Projects/ml_labs/lab5/.venv/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/ilyashulepov/Projects/ml_labs/lab5/.venv/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/Users/ilyashulepov/Projects/ml_labs/lab5/.venv/lib/python3.9/site-packages/torch/autograd/__init__.py:251: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Предсказание

In [2]:
# Получаем "raw" результат с полной инфой
raw_predictions, x = model.predict(val_dataloader, mode="raw", return_x=True)

NameError: name 'model' is not defined

Визуализация

In [ ]:
import matplotlib.pyplot as plt

idx = 0  # можно перебрать несколько

true = x["decoder_target"][idx].detach().cpu().numpy()
pred = raw_predictions["prediction"][idx].detach().cpu().numpy()

plt.plot(true, label="Истинное значение")
plt.plot(pred, label="Прогноз")
plt.title("Прогноз DeepAR")
plt.legend()
plt.show()
